In [1]:
import json
import pandas as pd
import numpy as np
import warnings
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import StandardScaler

# Filter out specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Read the JSON file
with open('courses.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract the data from the 'value' key
course_data = data['value']

# Convert course data to DataFrame
df_courses = pd.DataFrame(course_data)

# Select important attributes for embeddings
df_courses['embedding_text'] = df_courses.apply(
    lambda row: f"{row['NAME']} {row['PROFESSOR']} {row['DAYS']} {row['STARTTIME']} {row['ENDTIME']} {row['ROOM']} {row['INSTRUCTION_MODE']}",
    axis=1
)

# Generate embeddings for the selected attributes
model = SentenceTransformer('bert-base-nli-mean-tokens')
embeddings = model.encode(df_courses['embedding_text'].tolist())
embeddings = np.array(embeddings, dtype=np.float64)

# Define the test course to replace
test_course = "C S 329E Data Analytics TR 9:30 a.m.-11:00 a.m. Instructor XYZ"

# Function to recommend courses using Cosine Similarity
def recommend_cosine(course, embeddings, courses_df, top_n=7):
    course_embedding = model.encode([course])[0]
    similarities = cosine_similarity([course_embedding], embeddings)[0]
    indices = np.argsort(similarities)[::-1][1:top_n+1]
    return courses_df.iloc[indices]

# Function to recommend courses using K-Means Clustering
def recommend_kmeans(course, embeddings, courses_df, top_n=7, num_clusters=5):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    cluster_labels = kmeans.predict(embeddings)
    course_embedding = model.encode([course])[0]
    course_embedding = np.array(course_embedding, dtype=np.float64)  # Ensure dtype consistency
    cluster = kmeans.predict([course_embedding])[0]
    cluster_indices = np.where(cluster_labels == cluster)[0]
    similarities = cosine_similarity([course_embedding], embeddings[cluster_indices])[0]
    sorted_cluster_indices = cluster_indices[np.argsort(similarities)[::-1][1:top_n+1]]
    return courses_df.iloc[sorted_cluster_indices]

# Function to recommend courses using Autoencoder
def recommend_autoencoder(course, embeddings, courses_df, top_n=7):
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    course_embedding = scaler.transform(model.encode([course]))

    autoencoder = MLPRegressor(hidden_layer_sizes=(64, 32, 64), max_iter=5000, random_state=42)
    autoencoder.fit(embeddings_scaled, embeddings_scaled)

    course_embedding_pred = autoencoder.predict(course_embedding)
    similarities = cosine_similarity(course_embedding_pred, embeddings_scaled)[0]
    indices = np.argsort(similarities)[::-1][1:top_n+1]
    return courses_df.iloc[indices]


def recommend_dbscan(course, embeddings, courses_df, top_n=7, eps=0.5, min_samples=5):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(embeddings)
    
    # Find the cluster of the given course
    course_embedding = model.encode([course])[0]
    course_embedding = np.array(course_embedding, dtype=np.float64)
    cluster_labels = dbscan.fit_predict(np.vstack([embeddings, course_embedding]))

    # Check if the course is noise
    course_cluster = cluster_labels[-1]
    if course_cluster == -1:  # If the course is considered noise
        return pd.DataFrame()  # Or handle noise appropriately

    # Get indices of courses in the same cluster
    cluster_indices = np.where(clusters == course_cluster)[0]
    similarities = cosine_similarity([course_embedding], embeddings[cluster_indices])[0]
    sorted_cluster_indices = cluster_indices[np.argsort(similarities)[::-1][1:top_n+1]]
    return courses_df.iloc[sorted_cluster_indices]

# Print original course
print("Original course:", test_course)

# Generate recommendations using different methods
print("\nRecommendations using Cosine Similarity:")
recommendations_cosine = recommend_cosine(test_course, embeddings, df_courses)
print(recommendations_cosine[['NAME', 'DAYS', 'STARTTIME', 'ENDTIME']])

print("\nRecommendations using K-Means Clustering:")
recommendations_kmeans = recommend_kmeans(test_course, embeddings, df_courses)
print(recommendations_kmeans[['NAME', 'DAYS', 'STARTTIME', 'ENDTIME']])

print("\nRecommendations using Autoencoder:")
recommendations_autoencoder = recommend_autoencoder(test_course, embeddings, df_courses)
print(recommendations_autoencoder[['NAME', 'DAYS', 'STARTTIME', 'ENDTIME']])

print("\nRecommendations using DBSCAN:")
recommendations_dbscan = recommend_dbscan(test_course, embeddings, df_courses)
print(recommendations_dbscan[['NAME', 'DAYS', 'STARTTIME', 'ENDTIME']])


c:\Users\vingu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange



Original course: C S 329E Data Analytics TR 9:30 a.m.-11:00 a.m. Instructor XYZ

Recommendations using Cosine Similarity:
                                    NAME   DAYS STARTTIME ENDTIME
87   C S 329E ELEMENTS OF DATA ANALYTICS    TTH      8:30   10:00
95   C S 330E ELMNTS SOFTWARE ENGINEER I     MW      9:30   11:00
5   C S 309 AI LITERACY: ESSENT OF AI-WB    TTH      9:30   11:00
96   C S 330E ELMNTS SOFTWARE ENGINEER I     MW     11:00   12:30
77        C S 327E ELEMENTS OF DATABASES    TTH      9:30   11:00
28   C S 312 INTRODUCTION TO PROGRAMMING  MWF M      9:00   10:00
27   C S 312 INTRODUCTION TO PROGRAMMING  MWF M      9:00   10:00

Recommendations using K-Means Clustering:
                                    NAME   DAYS STARTTIME ENDTIME
87   C S 329E ELEMENTS OF DATA ANALYTICS    TTH      8:30   10:00
95   C S 330E ELMNTS SOFTWARE ENGINEER I     MW      9:30   11:00
5   C S 309 AI LITERACY: ESSENT OF AI-WB    TTH      9:30   11:00
96   C S 330E ELMNTS SOFTWARE ENGINEER I  

KeyError: "None of [Index(['NAME', 'DAYS', 'STARTTIME', 'ENDTIME'], dtype='object')] are in the [columns]"

In [ ]:
import numpy as np
import json
import warnings
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Filter out specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Sample course dataset
courses = [
    "Introduction to Computer Science",
    "Advanced Data Structures",
    "Machine Learning",
    "Database Systems",
    "Algorithms",
    "Artificial Intelligence",
    "Software Engineering",
    "Computer Networks",
    "Operating Systems",
    "Computer Graphics",
    "Data Mining"
]

# Load pre-trained model for embeddings
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Generate embeddings
embeddings = model.encode(courses)

# Ensure embeddings are of type float64 (double)
embeddings = np.array(embeddings, dtype=np.float64)

# Save embeddings and course names
with open('course_embeddings.json', 'w') as f:
    json.dump({
        'courses': courses,
        'embeddings': embeddings.tolist()
    }, f)

# Function to recommend courses based on cosine similarity
def recommend_cosine(course, embeddings, courses, top_n=3):
    course_embedding = model.encode([course])[0]
    similarities = cosine_similarity([course_embedding], embeddings)[0]
    indices = np.argsort(similarities)[::-1][1:top_n+1]
    return [courses[i] for i in indices]

# Function to recommend courses using clustering (K-Means)
def recommend_kmeans(course, embeddings, courses, top_n=3, num_clusters=5):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    cluster_labels = kmeans.predict(embeddings)
    course_embedding = model.encode([course])[0]
    course_embedding = np.array(course_embedding, dtype=np.float64)  # Ensure dtype consistency
    cluster = kmeans.predict([course_embedding])[0]
    cluster_indices = np.where(cluster_labels == cluster)[0]
    similarities = cosine_similarity([course_embedding], embeddings[cluster_indices])[0]
    sorted_cluster_indices = cluster_indices[np.argsort(similarities)[::-1][1:top_n+1]]
    return [courses[i] for i in sorted_cluster_indices]

# Function to recommend courses using neural network (Autoencoder)
def recommend_autoencoder(course, embeddings, courses, top_n=3):
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    course_embedding = scaler.transform(model.encode([course]))

    autoencoder = MLPRegressor(hidden_layer_sizes=(64, 32, 64), max_iter=5000, random_state=42)
    autoencoder.fit(embeddings_scaled, embeddings_scaled)

    course_embedding_pred = autoencoder.predict(course_embedding)
    similarities = cosine_similarity(course_embedding_pred, embeddings_scaled)[0]
    indices = np.argsort(similarities)[::-1][1:top_n+1]
    return [courses[i] for i in indices]

# Test the different methods
test_course = "Advanced Data Structures"

print("Original course:", test_course)
print("\nRecommendations using Cosine Similarity:")
print(recommend_cosine(test_course, embeddings, courses))

print("\nRecommendations using K-Means Clustering:")
print(recommend_kmeans(test_course, embeddings, courses))

print("\nRecommendations using Autoencoder:")
print(recommend_autoencoder(test_course, embeddings, courses))
